IN DEVELOPMENT

# Part 2: Training an RBM *with* a phase

## Getting Started

The following imports are needed to run this tutorial.

In [12]:
import torch
import numpy as np
import csv
import pickle

from qucumber.binary_rbm import BinaryRBM
from qucumber.quantum_reconstruction import QuantumReconstruction
from qucumber.complex_wavefunction import ComplexWavefunction

import qucumber.callbacks as MetricEvaluator 

import qucumber.utils.data as data              # for importing data
import qucumber.utils.cplx as cplx              # for complex algebra in torch
import qucumber.utils.unitaries as unitaries    # for importing unitary operators / gates

'''
#from rbm_tutorial import RBM_Module, BinomialRBM
import torch
#from observables_tutorial import TFIMChainEnergy, TFIMChainMagnetization
import numpy as np
import csv
%matplotlib inline
import sys
sys.path.append('../../qucumber/')
from complex_wavefunction import ComplexWavefunction
from quantum_reconstruction import QuantumReconstruction
sys.path.append('../../qucumber/utils/')
import unitaries
import utils.training_statistics as ts
import pickle
#import importlib.util
#%load_ext autoreload
#%autoreload 2
#%matplotlib notebook
'''

ModuleNotFoundError: No module named 'qucumber'

The _BinaryRBM_ class in *binary_rbm.py* contains the generic properties of an RBM with a binary visible and hidden layer (e.g. it's effective energy and sampling the hidden and visible layers). 

The actual quantum wavefunction reconstruction occurs in the _QuantumReconstruction_ class in *quantum_reconstruction.py*. A _QuantumReconstruction_ object is initialized with a neural network state (in this case, a *ComplexWavefunction* object).

*MetricEvaluator* in *callbacks.py* contains functions that allow the user to evaluate the quality of the training (i.e. based on the fidelity or KL divergence).

## Training

Let's go through training a complex wavefunction. To evaluate how the RBM is training, we will compute the full KL divergence and the fidelity between the true wavefunction of the system and the wavefunction the RBM reconstructs. We first need to load our training data and the true wavefunction of this system. However, we also need the corresponding file that contains all of the measurements that each site is in. The dummy dataset we will train our RBM on is a two qubit system who's wavefunction is $\psi =\left.\frac{1}{2}\right\vert+,+\rangle - \left.\frac{1}{2}\right\vert+,-\rangle + \left.\frac{i}{2}\right\vert-,+\rangle - \left.\frac{i}{2}\right\vert-,-\rangle$, where $+$ and $-$ represent spin-up and spin-down, respectively.

In [13]:
train_samples_path = 'qubits_train_samples.txt'
train_bases_path   = 'qubits_train_bases.txt'
bases_path         = 'qubits_bases.txt'
psi_path           = 'qubits_psi.txt'

train_samples,target_psi,train_bases,bases = data.load_data(train_samples_path, 
                                                            psi_path, 
                                                            train_bases_path, 
                                                            bases_path)

NameError: name 'data' is not defined

The following arguments are required to construct a **ComplexWavefunction** neural network state:

1. **A dictionary of unitary operators**. This will contain the (2 x 2) unitary matrices / gates.
2. **The number of visible units**. This is 2 for the case of our dataset.
3. **The number of hidden units in the hidden layer of the RBM**. This number is set to the number of visible units by default (10 in the case of our dataset).

In [7]:
unitary_dict = unitaries.create_dict()
'''If you would like to add your own quantum gates from your experiment to 
   "unitary_dict", do:
   unitary_dict = unitaries.create_dict(name='your_name', 
                                        unitary=torch.tensor([[real part], 
                                                              [imaginary part]], 
                                                             dtype=torch.double)
                                                             
   For example: 
   unitaries = unitary_library.create_dict(name='qucumber', 
                                           unitary=torch.tensor([ [[1.,0.],[0.,1.]] 
                                                                  [[0.,0.],[0.,0.]] ], 
                                                                dtype=torch.double))
                                                                                             
   By default, unitary_library.create_dict() contains the idenity matrix and the 
   hadamard and K gates with keys Z, X and Y, respectively.
'''

nv = train_samples.shape[-1]
nh = nv

nn_state = ComplexWavefunction(unitary_dict, num_visible=nv, num_hidden=nh)

NameError: name 'unitaries' is not defined

Now we can specify the parameters of the training process:

1. **epochs**: the number of epochs, i.e. training cycles that will be performed; 1000 should be fine
2. **batch_size**: the number of data points used in the positive phase of the gradient; we'll go with 100
3. **num_chains**: the number of data points used in the negative phase of the gradient. Keeping this larger than the *batch_size* is preferred; we'll go with 200
4. **CD**: the number of contrastive divergence steps; CD=1 seems to be good enough in most cases
5. **lr**: the learning rate; we will use a learning rate of 0.01 here
6. **log_every**: how often you would like the program to update you during the training; we choose 50 - that is, every 50 epochs the program will print out the fidelity. This parameter is required in the MetricEvaluator.

In [1]:
epochs     = 100
num_chains = 10
batch_size = 5
CD          = 5
lr         = 0.1
log_every  = 10

Once we initialize the parameters of the *ComplexWavefunction* and the *MetricEvaluator*, we can now begin training. Our *QuantumReconstruction* object, *qr* (see below), contains a function called *fit* that executes the training process.

In [1]:
nn_state.space = nn_state.generate_Hilbert_space(nv) # generate the entire visible space of the system.
callbacks      = [MetricEvaluator(log_every,{'Fidelity':ts.fidelity,'KL':ts.KL},target_psi=target_psi,bases=bases)]
z_samples      = data.extract_refbasis_samples(train_samples,train_bases) # required for the negative phase of the
                                                                          # gradient of the effective energy
    
qr = QuantumReconstruction(nn_state)

qr.fit(train_samples, epochs, batch_size, num_chains, CD,
       lr, input_bases=train_bases, progbar=False, callbacks = callbacks,
       z_samples = z_samples)

NameError: name 'nn_state' is not defined

### After Training 

After training your RBM, the *fit* function will have stored your trained weights and biases for the amplitude and the phase. Now, you have the option to generate new data from the trained RBM. The *rbm_real* object has a *sample* function that takes the following arguments.

1. The number of samples you wish to generate, *num_samples*.
2. The number of contrastive divergence steps performed to generate the samples, *k*.

In [17]:
num_samples = 50000
CD          = 200

nn_state.sample(CD)
samples = nn_state.visible_state

NameError: name 'ComplexWavefunction' is not defined

We can save the RBM parameters and the newly generated samples using the *save* function within the ComplexWavefunction object.

In [18]:
nn_state.save('saved_parameters.pkl', metadata={'Samples':samples})

NameError: name 'nn_state' is not defined